<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-2-Opti.-Mat./blob/main/P2_Tarea_2_Opti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 28.0 MB/s eta 0:00:00


In [17]:
import numpy as np
from gurobipy import*

In [111]:
#para esta pregunta se nos dan ciertos coeficientes a los cuales se les asignaran un valor
#Primero voy a plantear nuevamente el problema de optimización

#Definimos los datos que nos entrega el problema.
betha = np.array([[0,100,120,130,140,150,200,0]]).transpose() #Esto representa el costo de almacenaje

c = np.array([[0,10,25,30,0,0,20,0],[0,0,0,0,15,15,0,0],[0,0,0,0,0,15,0,0],
 [0,0,0,0,0,0,10,0],[0,0,0,0,0,0,0,25],[0,0,0,0,0,0,0,80],[0,0,0,0,0,0,0,25],  #representa el costo de ir desdela bodega i-esima hasta la j-esima, cij = 0 representa que esa ruta no existe
  [0,0,0,0,0,0,0,0]])

R = np.array([[0,500,350,500,0,0,600,0],[0,0,0,0,250,350,0,0],[0,0,0,0,0,350,0,0],
 [0,0,0,0,0,0,500,0],[0,0,0,0,0,0,0,800],[0,0,0,0,0,0,0,500],[0,0,0,0,0,0,0,500], #Representa cuanto se piuede transportar en la ruta desde i hasta j. Rij = 0 significa que esa ruta no está definida
  [0,0,0,0,0,0,0,0]])



In [112]:
#planteamos el problema de optimización
m = Model('Penalización')
n = 8

#definimos las variables, en este caso tenemos un problema de una sola variable, el cual no es lineal

x = m.addVars(n,n,vtype=GRB.CONTINUOUS, name = 'Cantidad_de_elemntos_por_bodega')

#restricciones:

m.addConstrs((sum(x[i,j] for i in range(n)) == 1000 for j in range (n)), name = 'Suma fila i-esima en X')
m.addConstrs(((x[i,j] -r[i,j]) <=0 for i in range (n) for j in range (n)), name = 'Carga ruta ij')

m.setObjective(
    quicksum(c[i, j] * x[i, j] * x[i, j] for i in range(n) for j in range(n)) +
    quicksum(betha[i] * quicksum(x[i, j] for j in range(n)) for i in range(n)),
    GRB.MINIMIZE)

m.optimize()

<ipython-input-112-0102c07ca69c>:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  quicksum(betha[i] * quicksum(x[i, j] for j in range(n)) for i in range(n)),
<ipython-input-112-0102c07ca69c>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  quicksum(c[i, j] * x[i, j] * x[i, j] for i in range(n) for j in range(n)) +
<ipython-input-112-0102c07ca69c>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  m.setObjective(


GurobiError: Unable to convert argument to an expression